In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')

print("✅ Libraries imported successfully")

## 1. Load Raw Data

In [ ]:
# Load the dataset
data_path = '../data/data.csv'
df_raw = pd.read_csv(data_path)

print(f"Dataset shape: {df_raw.shape}")
print(f"\nColumns: {list(df_raw.columns)}")
print(f"\nFirst 5 rows:")
df_raw.head()

In [ ]:
# Display data types and info
print("Data Types:")
print(df_raw.dtypes)
print("\n" + "="*60)
print("Dataset Info:")
df_raw.info()

## 2. Data Cleaning

In [ ]:
# Create a copy for cleaning
df = df_raw.copy()

# Convert Time column to datetime
df['Time'] = pd.to_datetime(df['Time'])
df['Year'] = df['Time'].dt.year

print(f"Time range: {df['Year'].min()} to {df['Year'].max()}")
print(f"\nUnique countries: {df['Country'].nunique()}")
print(df['Country'].unique())

In [ ]:
# Clean Amount column - convert to numeric
# First, check what type of values we have
print("Sample Amount values before cleaning:")
print(df['Amount'].head(20))
print(f"\nAmount data type: {df['Amount'].dtype}")

# Convert to numeric, coercing errors to NaN
df['Amount'] = pd.to_numeric(df['Amount'], errors='coerce')

print(f"\n✅ Amount column converted to numeric")
print(f"Missing values in Amount: {df['Amount'].isna().sum()} ({df['Amount'].isna().sum()/len(df)*100:.2f}%)")

In [ ]:
# Standardize indicator names (remove trailing spaces and duplicates)
df['Indicator'] = df['Indicator'].str.strip()

# Consolidate duplicate indicators
indicator_mapping = {
    'GDP per capita': 'GDP per Capita',
    'GDP per Capita ': 'GDP per Capita',
    'Inflation Rate ': 'Inflation Rate',
    'Food Inflation ': 'Food Inflation',
    'Food Inflation YoY': 'Food Inflation'
}

df['Indicator'] = df['Indicator'].replace(indicator_mapping)

print("Unique indicators after standardization:")
print(df['Indicator'].value_counts())

## 3. Missing Value Analysis

In [ ]:
# Analyze missing values
print("Missing values by column:")
missing_summary = pd.DataFrame({
    'Missing_Count': df.isnull().sum(),
    'Missing_Percent': (df.isnull().sum() / len(df) * 100).round(2)
}).sort_values('Missing_Count', ascending=False)

print(missing_summary[missing_summary['Missing_Count'] > 0])

In [ ]:
# Check missing values by indicator
missing_by_indicator = df.groupby('Indicator')['Amount'].apply(
    lambda x: (x.isna().sum(), len(x), x.isna().sum()/len(x)*100)
).apply(pd.Series)

missing_by_indicator.columns = ['Missing', 'Total', 'Percent_Missing']
missing_by_indicator = missing_by_indicator.sort_values('Percent_Missing', ascending=False)

print("\nMissing values by indicator:")
print(missing_by_indicator[missing_by_indicator['Missing'] > 0])

In [ ]:
# Visualize data coverage by country and indicator
coverage = df.groupby(['Country', 'Indicator']).agg({
    'Amount': lambda x: x.notna().sum(),
    'Year': 'count'
}).reset_index()

coverage['Coverage_Percent'] = (coverage['Amount'] / coverage['Year'] * 100).round(1)

plt.figure(figsize=(14, 8))
pivot_coverage = coverage.pivot(index='Country', columns='Indicator', values='Coverage_Percent')
sns.heatmap(pivot_coverage, annot=False, cmap='YlGnBu', cbar_kws={'label': 'Coverage %'})
plt.title('Data Coverage by Country and Indicator (%)', fontsize=14, fontweight='bold')
plt.xlabel('Indicator', fontsize=11)
plt.ylabel('Country', fontsize=11)
plt.xticks(rotation=90, ha='right')
plt.tight_layout()
plt.show()

print("\n✅ Data coverage visualization complete")

## 4. Create Panel Dataset (Wide Format)

In [ ]:
# Pivot data to wide format: rows = (Country, Year), columns = Indicators
df_panel = df.pivot_table(
    index=['Country', 'Country Code', 'Year'],
    columns='Indicator',
    values='Amount',
    aggfunc='first'  # In case of duplicates, take first value
).reset_index()

print(f"Panel dataset shape: {df_panel.shape}")
print(f"\nCountries: {df_panel['Country'].nunique()}")
print(f"Years: {df_panel['Year'].min()} to {df_panel['Year'].max()}")
print(f"Total observations: {len(df_panel)}")

df_panel.head(10)

In [ ]:
# Rename columns to snake_case for easier access
df_panel.columns = [col.lower().replace(' ', '_').replace('/', '_').replace('(', '').replace(')', '') for col in df_panel.columns]

print("Column names after standardization:")
print(list(df_panel.columns))

In [ ]:
# Check for duplicate indicator columns and consolidate
# Check column names
print("Columns with potential duplicates:")
cols = list(df_panel.columns)
for col in cols:
    if 'gdp_per_capita' in col:
        print(f"  - {col}")

## 5. Data Quality Summary

In [ ]:
# Calculate completeness for each country
country_completeness = df_panel.groupby('country').apply(
    lambda x: x.notna().sum().sum() / (len(x) * (len(x.columns) - 3)) * 100
).sort_values(ascending=False)

print("Data completeness by country (%):\n")
for country, completeness in country_completeness.items():
    print(f"{country:20s}: {completeness:5.1f}%")

# Visualize
plt.figure(figsize=(10, 6))
country_completeness.plot(kind='barh', color='steelblue')
plt.xlabel('Completeness (%)', fontsize=11)
plt.ylabel('Country', fontsize=11)
plt.title('Data Completeness by Country', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Identify countries with best coverage for focus analysis
top_coverage_countries = country_completeness.head(6).index.tolist()

print("\n🎯 Recommended focus countries (best data coverage):")
for i, country in enumerate(top_coverage_countries, 1):
    print(f"{i}. {country} ({country_completeness[country]:.1f}% complete)")

## 6. Export Cleaned Data

In [ ]:
# Export cleaned panel dataset
output_path = '../data/cleaned_panel_data.csv'
df_panel.to_csv(output_path, index=False)

print(f"✅ Cleaned panel data saved to: {output_path}")
print(f"   Shape: {df_panel.shape}")
print(f"   Size: {Path(output_path).stat().st_size / 1024:.1f} KB")

In [ ]:
# Also save the long-format cleaned data
output_long_path = '../data/cleaned_long_data.csv'
df.to_csv(output_long_path, index=False)

print(f"✅ Cleaned long-format data saved to: {output_long_path}")
print(f"   Shape: {df.shape}")

## 7. Summary Statistics

In [ ]:
# Display summary statistics for key indicators
key_indicators = [
    'government_debt', 'budget_deficit_surplus', 'nominal_gdp', 'real_gdp',
    'gdp_growth_rate', 'inflation_rate', 'revenue', 'expenditure'
]

available_indicators = [col for col in key_indicators if col in df_panel.columns]

if available_indicators:
    print("Summary statistics for key indicators:\n")
    print(df_panel[available_indicators].describe().round(2))
else:
    print("Key indicators not found with expected names. Available columns:")
    print([col for col in df_panel.columns if col not in ['country', 'country_code', 'year']])

## ✅ Data Preparation Complete!

### Next Steps:
1. **Exploratory Data Analysis** (`01_eda.ipynb`)
2. **Feature Engineering** (`02_feature_engineering.ipynb`)
3. **ML Model Development** (`03_ml_debt_crisis.ipynb`)

### Outputs:
- `cleaned_panel_data.csv` - Wide format (country-year panel)
- `cleaned_long_data.csv` - Long format (one row per observation)

### Key Findings:
- Dataset covers 14 African countries from 1960-2025
- 27 unique indicators after standardization
- Top 6 countries recommended for detailed analysis based on data completeness